<a href="https://colab.research.google.com/github/MRVithu/Sentiment-Analysis-for-Movie-Reviews/blob/main/Sentiment_Analysis_for_Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Extracting tar.gz file
import tarfile

tar = tarfile.open("/content/review_polarity.tar.gz", 'r:gz')
tar.extractall()
tar.close()

In [2]:
#To store review(Text) and sentiments
review_arr = []
status_arr = []

In [ ]:
#Read tha files and store in array
import os
import io

dir = "/content/txt_sentoken/"
sentiment_arr = ['neg', 'pos']
i = 0
for status in sentiment_arr:
    text_files = os.listdir(dir+status)
  
    for file_name in text_files:
        i=i+1
        file_dir = dir+status+'/'+file_name
        text_data = list()
        try:
            file = open(file_dir, "r")
            file_content = file.read()
            status_arr.append(status)
            text_data = file_content.split('\n')
            text_data = list(filter(None, text_data))          
            review_arr.append(' '.join(text_data[:]))
            print('success --> ' + str(i))
        except OSError:
            print('fail --> ' + str(i))



In [6]:
# import regex(re) and NLP(nltk) packages 
import re
import nltk

In [7]:
# Data Cleaning
processed_features = []

for sentence in range(0, len(review_arr)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(review_arr[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

In [8]:
#import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
# Using TfidfVectorizer to convert text review into TF-IDF review vectors.
vectorizer = TfidfVectorizer (ngram_range=(1, 2),max_features=2500, min_df=7, max_df=0.8)
processed_features = vectorizer.fit_transform(processed_features).toarray()

In [10]:
#Dividing Data into Training and Test Sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(processed_features, status_arr, test_size=0.2, random_state=0)

In [11]:
#Training the Model
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [12]:
#Making Predictions and Evaluating the Model
predictions = text_classifier.predict(X_test)

In [15]:
# Evaluate the performance of the model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[172  28]
 [ 44 156]]
              precision    recall  f1-score   support

         neg       0.80      0.86      0.83       200
         pos       0.85      0.78      0.81       200

    accuracy                           0.82       400
   macro avg       0.82      0.82      0.82       400
weighted avg       0.82      0.82      0.82       400

0.82
